In [2]:
import pandas as pd
import numpy as np
from pulp import *

In [3]:
ps= pd.read_excel('Oats_prod_schedule.xlsx')

In [4]:
ps


,Month,demand,fixed_cost,var_cost,storage_cost,capacity
0,Jan,3000,3000,70,1.0,10000
1,Feb,4000,3000,50,1.0,10000
2,Mar,2500,3000,80,1.0,10000
3,Apr,4000,3000,60,1.0,10000
4,May,6000,3000,75,1.0,10000
5,Jun,7000,3000,60,1.0,10000
6,Jul,8200,2000,20,1.2,8000
7,Aug,5000,2000,40,1.2,8000
8,Sep,7000,2000,45,1.2,8000
9,Oct,4000,2000,50,1.2,7000


In [4]:
ps.shape

(12, 6)

In [5]:
ps['sr']=range(1,13)
ps=ps.set_index('sr')
ps

,Month,demand,fixed_cost,var_cost,storage_cost,capacity
sr,,,,,,
1,Jan,3000,3000,70,1.0,10000
2,Feb,4000,3000,50,1.0,10000
3,Mar,2500,3000,80,1.0,10000
4,Apr,4000,3000,60,1.0,10000
5,May,6000,3000,75,1.0,10000
6,Jun,7000,3000,60,1.0,10000
7,Jul,8200,2000,20,1.2,8000
8,Aug,5000,2000,40,1.2,8000
9,Sep,7000,2000,45,1.2,8000


In [7]:
model=LpProblem('scheduling',LpMinimize)

In [8]:
binary=LpVariable.dicts('binary',[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],0,None,'Binary')

In [9]:
production=LpVariable.dicts('production',[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],0,None,'Integer')

In [10]:
inventory=LpVariable.dicts('inventory',[0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],0,None,'Integer')

In [11]:
inventory[0]=1000

In [12]:
ps.head(3)

,Month,demand,fixed_cost,var_cost,storage_cost,capacity
sr,,,,,,
1,Jan,3000,3000,70,1.0,10000
2,Feb,4000,3000,50,1.0,10000
3,Mar,2500,3000,80,1.0,10000


In [13]:
time=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

#Constraints

In [14]:
for x in time:
    model+=production[x]<=binary[x]*ps.loc[x,'capacity']
    

In [15]:
for x in time:
    model+=(production[x]+inventory[x-1])-inventory[x]>=ps.loc[x,'demand']

#Objective Function

In [16]:
model+=lpSum(production[x]*ps.loc[x,'var_cost'] + inventory[x]*ps.loc[x,'storage_cost']+
             binary[x]*ps.loc[x,'fixed_cost'] for x in time)

In [17]:
model.solve()

1

In [17]:
for i in binary:
    print (f'binary are {binary[i].varValue}')

binary are 1.0
binary are 1.0
binary are 0.0
binary are 1.0
binary are 0.0
binary are 1.0
binary are 1.0
binary are 1.0
binary are 1.0
binary are 0.0
binary are 1.0
binary are 1.0


In [18]:
for i in production:
    print (f'production are {production[i].varValue}')

production are 2000.0
production are 10000.0
production are 0.0
production are 6500.0
production are 0.0
production are 7200.0
production are 8000.0
production are 8000.0
production are 8000.0
production are 0.0
production are 8000.0
production are 500.0


In [19]:
for i in range(1,13):
    print (f'inventory is {inventory[i].varValue}')

inventory is 0.0
inventory is 6000.0
inventory is 3500.0
inventory is 6000.0
inventory is 0.0
inventory is 200.0
inventory is 0.0
inventory is 3000.0
inventory is 4000.0
inventory is 0.0
inventory is 3000.0
inventory is 0.0


In [18]:
for v in model.variables():
    print(v, v.varValue)

binary_1 1.0
binary_10 0.0
binary_11 1.0
binary_12 1.0
binary_2 1.0
binary_3 0.0
binary_4 1.0
binary_5 0.0
binary_6 1.0
binary_7 1.0
binary_8 1.0
binary_9 1.0
inventory_1 0.0
inventory_10 0.0
inventory_11 3000.0
inventory_12 0.0
inventory_2 6000.0
inventory_3 3500.0
inventory_4 6000.0
inventory_5 0.0
inventory_6 200.0
inventory_7 0.0
inventory_8 3000.0
inventory_9 4000.0
production_1 2000.0
production_10 0.0
production_11 8000.0
production_12 500.0
production_2 10000.0
production_3 0.0
production_4 6500.0
production_5 0.0
production_6 7200.0
production_7 8000.0
production_8 8000.0
production_9 8000.0


In [19]:
optimization_data=pd.DataFrame({'Demand':ps['demand'],'Inventory':[inventory[i].varValue for i in range(1,13)],
    'Production qty':[production[i].varValue for i in production],
    'Binary':[binary[i].varValue for i in binary]})

In [20]:
optimization_data

,Demand,Inventory,Production qty,Binary
sr,,,,
1,3000,0.0,2000.0,1.0
2,4000,6000.0,10000.0,1.0
3,2500,3500.0,0.0,0.0
4,4000,6000.0,6500.0,1.0
5,6000,0.0,0.0,0.0
6,7000,200.0,7200.0,1.0
7,8200,0.0,8000.0,1.0
8,5000,3000.0,8000.0,1.0
9,7000,4000.0,8000.0,1.0
